In [1]:
import pickle

In [2]:
from cube_fil_finder.util import moments
from cube_fil_finder.galfa import galfa_const
from cube_fil_finder.galfa import galfa_util
from cube_fil_finder.vis import node_vis
from cube_fil_finder.util import cube_moments
from cube_fil_finder.util import cube_util
from cube_fil_finder.util import widths

import os
import logging
import pickle
import numpy as np
from matplotlib import pyplot as plt
from matplotlib import patches as mpatches

In [3]:
trees = pickle.load(open('../../pickled_dicts/fifth_batch/filtered_trees_1_6.p'))

In [4]:
all_trees_mask_orientation_info = pickle.load(open('../../pickled_dicts/fifth_batch/all_trees_mask_orientation_info.p'))

In [5]:
logging.getLogger().setLevel('DEBUG')

In [6]:
from astropy.io import fits
from astropy import units as u
# example header
hdr = fits.getheader('/Volumes/LarryExternal1/Research_2017/GALFA_slices_backup/umask_gaussian_30/GALFA_HI_W_S0955_V-050.4kms_umask.fits')

In [7]:
trees_data_1_6 = {}

In [13]:
all_trees = {k: v for k, v in trees.items() if v.getTreeMaskedArea2D() >= 20000}

In [14]:
[v.getTreeMaskedArea2D() for k, v in all_trees.items()]

[21052,
 25020,
 30788,
 41654,
 21015,
 42888,
 23875,
 94377,
 25646,
 115157,
 21216,
 20724,
 30215]

In [15]:
for i, k in enumerate(all_trees):
    tree = all_trees[k]
    logging.debug('on key: {0}'.format(k))

    if all_trees_mask_orientation_info.get(k, [1])[-1] > moments.ROUNDNESS_AR_CONVERSION['1_6']:
        logging.debug('skipping b/c {0} didn\'t make the base ~1:6 aspect ratio cut'.format(k))
        continue
    
    corners = tree.root_node.corners
    if (
        corners[1][0] > galfa_const.GALFA_Y_STEPS - 10 or 
        corners[1][1] > galfa_const.GALFA_X_STEPS - 10 or 
        corners[0][0] < 10 or 
        corners[0][1] < 10):
        logging.debug('skipping b/c {0} is too close to the edge'.format(k))
        continue 

    # 2d binary numpy mask
    tree_mask = tree.root_node.mask
    # number of pixels masked
    tree_size = tree.root_node.masked_area_size
    # velocity span in km/s from number of channels
    tree_v_span = tree.length * galfa_const.GALFA_W_SLICE_SEPARATION
    tree_starting_v_index = tree.root_v_slice

    # "average velocity" use mean/med/max of moment 1 map instead
    tree_avg_v = galfa_util.galfa_v_lookup_from_index(tree_starting_v_index + tree_v_span / 2.)
    
    # centroid and roundness measurement of the mask
    x_bar, y_bar, theta_1, theta_2, tree_roundness = all_trees_mask_orientation_info.get(k, [np.inf] * 5)
    # conversion of the centroid index into ra&dec and l&b
    tree_ra, tree_dec = galfa_util.galfa_index_to_radecs(x_bar, y_bar)
    tree_l, tree_b = galfa_util.galfa_index_to_lb(x_bar, y_bar)
    # convert the corners into ra&dec
    xs = [tree.root_node.corners[0][1], tree.root_node.corners[1][1]]
    ys = [tree.root_node.corners[0][0], tree.root_node.corners[1][0]]
    corners_ra, corners_dec = galfa_util.galfa_index_to_radecs(xs, ys)    
    
    # list of cut umask slices
    tree_data_cube = galfa_util.get_galfa_data_cube_from_tree(tree)
    # list of cut raw slices
    tree_data_cube_raw = galfa_util.get_galfa_data_cube_from_tree(tree, cube_type='raw')

    # moment 0 from the umask slices
    full_moment_0_map = np.nan_to_num(cube_moments.moment_0_from_cube(tree_data_cube))
    # moment 0 from the raw slices
    full_moment_0_map_raw = np.nan_to_num(cube_moments.moment_0_from_cube(tree_data_cube_raw))
    # unsharp masking the moment 0 from the raw slices
    full_moment_0_map_umask = cube_util.umask(full_moment_0_map_raw, radius=30, filter_opt='gaussian')

    # moment 0, 1, 2, column maps with pixels not in the mask NaN'd out
    moment_0_map = cube_moments.moment_0_from_cube(tree_data_cube, mask=tree_mask)
    moment_0_map_raw = full_moment_0_map_umask.copy()
    moment_0_map_raw[np.where(tree_mask == 0)] = np.nan
    moment_1_map = cube_moments.moment_1_from_cube(tree_data_cube, tree_starting_v_index, tree.length, mask=tree_mask)
    moment_2_map = cube_moments.moment_2_from_cube(tree_data_cube, tree_starting_v_index, tree.length, mask=tree_mask)
    column_density_map = cube_moments.column_density_from_moment_0_map(moment_0_map)
    column_density_map_raw = cube_moments.column_density_from_moment_0_map(moment_0_map_raw)

    # width fits, errors, and chisq from both the (umask-->moment0) and the (raw-->moment0-->umask) moment0s
    #width_fit, width_fit_err, chisq = widths.get_width_fit_filfind(full_moment_0_map, tree, hdr)
    #width_fit_raw, width_fit_err_raw, chisq_raw = widths.get_width_fit_filfind(full_moment_0_map_umask, tree, hdr)

    tree_data = {}
    tree_data['key_num'] = i
    tree_data['roundness'] = tree_roundness
    tree_data['size'] = tree_size
    tree_data['ra'] = tree_ra
    tree_data['dec'] = tree_dec
    tree_data['l'] = tree_l
    tree_data['b'] = tree_b
    tree_data['velocity_span'] = tree_v_span
    tree_data['starting_velocity'] = galfa_util.galfa_v_lookup_from_index(tree.root_v_slice)
    tree_data['average_velocity'] = tree_avg_v
    tree_data['moment_0_peak'] = np.nanmax(moment_0_map)
    tree_data['moment_0_peak_raw']= np.nanmax(moment_0_map_raw)
    tree_data['moment_0_mean'] = np.nanmean(moment_0_map)
    tree_data['moment_0_mean_raw']= np.nanmean(moment_0_map_raw)
    tree_data['moment_0_median'] = np.nanmedian(moment_0_map)
    tree_data['moment_0_median_raw']= np.nanmedian(moment_0_map_raw)
    tree_data['moment_1_mean'] = np.nanmean(moment_1_map)
    tree_data['moment_1_median'] = np.nanmedian(moment_1_map)
    tree_data['moment_2_peak'] = np.nanmax(moment_2_map)
    tree_data['moment_2_mean'] = np.nanmean(moment_2_map)
    tree_data['moment_2_median'] = np.nanmedian(moment_2_map)
    tree_data['column_density_peak'] = np.nanmax(column_density_map)
    tree_data['column_density_peak_raw'] = np.nanmax(column_density_map_raw)
    tree_data['column_density_mean'] = np.nanmean(column_density_map)
    tree_data['column_density_mean_raw'] = np.nanmean(column_density_map_raw)
    tree_data['column_density_median'] = np.nanmedian(column_density_map)
    tree_data['column_density_median_raw'] = np.nanmedian(column_density_map_raw)
    tree_data['width_fit'] = None
    tree_data['width_fit_raw'] = None
    tree_data['width_err'] = None
    tree_data['width_err_raw'] = None
    tree_data['width_chisq'] = None
    tree_data['width_chisq_raw'] = None
    
    trees_data_1_6[k] = tree_data
    

    fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize=(8,5))

    maps = [column_density_map, full_moment_0_map_umask]

    for i, ax, in enumerate([ax1, ax2]):
        cutoff = np.percentile(maps[i], 5)
        cutoff2 = np.percentile(maps[i], 95)
        im = ax.imshow(maps[i].clip(cutoff, cutoff2), origin='lower', cmap='binary') #imshow does m by n
        ax.contour(tree_mask, alpha=.5, colors='red', linewidths=.1)

        ax.set_xticks([0, tree.root_node.corners[1][1]-tree.root_node.corners[0][1]])
        ax.set_xticklabels(np.round(corners_ra,2))
        ax.set_yticks([0, tree.root_node.corners[1][0]-tree.root_node.corners[0][0]])
        ax.set_yticklabels(np.round(corners_dec,2))
        ax.set_xlabel('RA')
        ax.set_ylabel('DEC')

        if i == 0:
            ax.set_title('column from umask30')
        elif i == 1:
            ax.set_title('raw moment 0 (w/ unsharp mask)')
        fig.colorbar(im, ax=ax, shrink=.8)
    

    fig.suptitle('{0} ({1} channels)'.format(k, tree.length))
    fig.tight_layout()
        
    if tree_roundness <= moments.ROUNDNESS_AR_CONVERSION['1_12']:
        save_path = '../../vis/column_plots_fifth_batch/1_12/{0}.pdf'.format(k)
    elif tree_roundness <= moments.ROUNDNESS_AR_CONVERSION['1_10']:
        save_path = '../../vis/column_plots_fifth_batch/1_10/{0}.pdf'.format(k)
    elif tree_roundness <= moments.ROUNDNESS_AR_CONVERSION['1_8']:
        save_path = '../../vis/column_plots_fifth_batch/1_8/{0}.pdf'.format(k)
    else:
        save_path = '../../vis/column_plots_fifth_batch/1_6/{0}.pdf'.format(k)
    
    logging.debug('saving figure at {0}'.format(save_path))
    fig.savefig(save_path)
    fig.clf()
    plt.close()
    plt.close('all')

DEBUG:root:on key: 422_964_25
DEBUG:root:doing gaussian umask filter
DEBUG:root:saving figure at ../../vis/column_plots_fifth_batch/1_6/422_964_25.pdf
DEBUG:root:on key: 24903_1025_0
DEBUG:root:doing gaussian umask filter
DEBUG:root:saving figure at ../../vis/column_plots_fifth_batch/1_10/24903_1025_0.pdf
DEBUG:root:on key: 537_1041_3
DEBUG:root:doing gaussian umask filter
DEBUG:root:saving figure at ../../vis/column_plots_fifth_batch/1_6/537_1041_3.pdf
DEBUG:root:on key: 41615_1042_0
DEBUG:root:doing gaussian umask filter
DEBUG:root:saving figure at ../../vis/column_plots_fifth_batch/1_6/41615_1042_0.pdf
DEBUG:root:on key: 19662_1016_0
DEBUG:root:doing gaussian umask filter
DEBUG:root:saving figure at ../../vis/column_plots_fifth_batch/1_12/19662_1016_0.pdf
DEBUG:root:on key: 37552_1054_0
DEBUG:root:doing gaussian umask filter
DEBUG:root:saving figure at ../../vis/column_plots_fifth_batch/1_8/37552_1054_0.pdf
DEBUG:root:on key: 848_1017_0
DEBUG:root:doing gaussian umask filter
DEBUG:r

In [17]:
pickle.dump(trees_data_1_6, open('../../pickled_dicts/fifth_batch/trees_data_1_6_large.p', 'wb'))